In [33]:
pip install scikit-multilearn

  Obtaining dependency information for scikit-multilearn from https://files.pythonhosted.org/packages/bb/1f/e6ff649c72a1cdf2c7a1d31eb21705110ce1c5d3e7e26b2cc300e1637272/scikit_multilearn-0.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 834.8 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from skmultilearn.adapt import MLkNN 
from sklearn.metrics import hamming_loss, accuracy_score 

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [47]:
# DATA Exploration

In [48]:
# import data
df= pd.read_csv('Symptom2Disease.csv')
df.head()

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1200 non-null   int64 
 1   label       1200 non-null   object
 2   text        1200 non-null   object
dtypes: int64(1), object(2)
memory usage: 28.3+ KB


In [50]:
df.drop(["Unnamed: 0"], axis = 1, inplace=True)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1200 non-null   object
 1   text    1200 non-null   object
dtypes: object(2)
memory usage: 18.9+ KB


In [52]:
df.head()

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [53]:
# Check for data classes
df['label'].nunique()

24

In [54]:
unique_values = df['text'].unique()
unique_count = df['text'].nunique()

#print("Unique values:", unique_values)
print("Count of unique values:", unique_count)

Count of unique values: 1153


In [55]:
a = df['label'].unique()
print(a)
print(type(a))

['Psoriasis' 'Varicose Veins' 'Typhoid' 'Chicken pox' 'Impetigo' 'Dengue'
 'Fungal infection' 'Common Cold' 'Pneumonia' 'Dimorphic Hemorrhoids'
 'Arthritis' 'Acne' 'Bronchial Asthma' 'Hypertension' 'Migraine'
 'Cervical spondylosis' 'Jaundice' 'Malaria' 'urinary tract infection'
 'allergy' 'gastroesophageal reflux disease' 'drug reaction'
 'peptic ulcer disease' 'diabetes']
<class 'numpy.ndarray'>


In [56]:
df.shape

(1200, 2)

In [57]:
df.duplicated().sum()

47

In [58]:
df[df.duplicated].head()

,label,text
163,Chicken pox,I'm feeling fatigued and have no energy. I can...
387,Common Cold,I've been quite exhausted and ill. My throat h...
430,Pneumonia,"I have a really high fever, and I have problem..."
433,Pneumonia,I'm having a hard time breathing and I feel re...
438,Pneumonia,"Lately I've been experiencing chills, fatigue,..."


In [59]:
df.drop_duplicates(inplace= True)

In [60]:
df.shape

(1153, 2)

In [61]:
df['label'].value_counts()

label
Psoriasis                          50
Varicose Veins                     50
peptic ulcer disease               50
drug reaction                      50
allergy                            50
urinary tract infection            50
Hypertension                       50
diabetes                           50
Fungal infection                   50
Dengue                             50
Impetigo                           50
Typhoid                            50
Common Cold                        49
Cervical spondylosis               49
Chicken pox                        49
Bronchial Asthma                   49
gastroesophageal reflux disease    48
Pneumonia                          47
Migraine                           47
Arthritis                          46
Acne                               46
Malaria                            44
Dimorphic Hemorrhoids              41
Jaundice                           38
Name: count, dtype: int64

In [62]:
df.head()

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."


In [63]:
df.tail()

,label,text
1195,diabetes,I'm shaking and trembling all over. I've lost ...
1196,diabetes,"Particularly in the crevices of my skin, I hav..."
1197,diabetes,I regularly experience these intense urges and...
1198,diabetes,"I have trouble breathing, especially outside. ..."
1199,diabetes,I constantly sneeze and have a dry cough. My i...


In [64]:
df.isnull().sum() # to check null value

label    0
text     0
dtype: int64

In [65]:
#df.drop('label',axis=1, inplace=True)

## Data Preprocessing

In [66]:
df['text']

0       I have been experiencing a skin rash on my arm...
1       My skin has been peeling, especially on my kne...
2       I have been experiencing joint pain in my fing...
3       There is a silver like dusting on my skin, esp...
4       My nails have small dents or pits in them, and...
                              ...                        
1195    I'm shaking and trembling all over. I've lost ...
1196    Particularly in the crevices of my skin, I hav...
1197    I regularly experience these intense urges and...
1198    I have trouble breathing, especially outside. ...
1199    I constantly sneeze and have a dry cough. My i...
Name: text, Length: 1153, dtype: object

In [67]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
data = df
if not sys.warnoptions:
    warnings.simplefilter("ignore")
def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
data['text'] = data['text'].str.lower()
data['text'] = data['text'].apply(cleanHtml)
data['text'] = data['text'].apply(cleanPunc)
data['text'] = data['text'].apply(keepAlpha)

#Stop word Removal

stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)
data['text'] = data['text'].apply(removeStopWords)


In [68]:
#stemming
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence
data['text'] = data['text'].apply(stemming)

In [69]:
data

,label,text
0,Psoriasis,experienc skin rash arm leg torso past week re...
1,Psoriasis,skin peel especi knee elbow scalp peel often a...
2,Psoriasis,experienc joint pain finger wrist knee pain of...
3,Psoriasis,silver like dust skin especi lower back scalp ...
4,Psoriasis,nail small dent pit often feel inflammatori te...
...,...,...
1195,diabetes,im shake trembl ive lost sens tast smell im ex...
1196,diabetes,particular crevic skin skin rash irrit skin br...
1197,diabetes,regular experi intens urg want urin frequent f...
1198,diabetes,troubl breath especi outsid start feel hot sta...


In [70]:
data['text'].isnull().sum() # to check null value

0

In [202]:
#text_df = data.drop('label', axis=1)

In [71]:
data.to_csv('cleaned_data_nltk', sep=',')